In [1]:
%gui qt5
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [44]:
from matplotlib import pyplot as plt
from matplotlib.widgets import MultiCursor
import numpy as np
import xarray as xr
import pandas as pd

In [3]:
from theplotter.graph import nodes
from theplotter.plugins.targets import mpl

In [11]:
class NpArraySource(nodes.Source):
    def __init__(self, arr, **kwargs):
        super().__init__(**kwargs)
        self.dataset = arr
        
    def run(self):
        self.need_update = False
        

class MultiArray(nodes.Node):
    def __init__(self, **arrays):
        super().__init__()
        
        self.names = []
        for k, v in arrays.items():
            src = NpArraySource(v, name=k)
            self.datasources.append(src)
            src.add_target_if_absent(self)
            self.names.append(k)
            
    def set_array(self, name, value):
        src = self.datasources[self.names.index(name)]
        src.dataset = value
        self.need_update = True
            
    def __getitem__(self, item):
        return self.datasources[self.names.index(item)]
    
    def __setitem__ (self, key, item):
        self.set_array(key, item)
        
class XArray(nodes.Source):
    pass

In [7]:
x = np.linspace(0,10,101)
z = np.cos(x)
xzsrc = MultiArray(x=x, z=z)

In [8]:
w = mpl.MPLWindow(1,1)
plt = mpl.lineplot(datasources=[xzsrc['x'], xzsrc['z']], plottarget=w[0,0])

In [9]:
xzsrc['z'] = z**2

In [ ]:
zsrc.dataset = z**2
zsrc.need_update = True
# plt.play()
# zsrc.run()

In [41]:
x = np.arange(5)
y = np.arange(3)
xx, yy = np.meshgrid(x, y, indexing='ij')
zz = xx*yy**2

In [42]:
xcol = xx.reshape(-1)
ycol = yy.reshape(-1)
zcol = zz.reshape(-1)
zcol2 = np.array([z * np.arange(3) for z in zcol])

In [46]:
df = pd.DataFrame(dict(x=xcol, y=ycol, z=zcol))
df

,x,y,z
0,0,0,0
1,0,1,0
2,0,2,0
3,1,0,0
4,1,1,1
5,1,2,4
6,2,0,0
7,2,1,2
8,2,2,8
9,3,0,0


In [35]:
np.array(3 * [xcol]).T.reshape(-1).reshape(3,5,3)

array([[[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4]],

       [[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4]],

       [[0, 0, 0],
        [1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4]]])

In [66]:
def grid_to_table(data, data_name='data', coords=[], order='inverted'):
    shp = data.shape
    data_col = data.reshape(-1)
    
    dif = len(shp)-len(coords)
    for i in range(dif):
        coords.append((f'axis {i}', np.arange(shp[i-dif])))
        
    print(coords)

In [67]:
grid_to_table(zz, data_name='z', coords=[])

[('axis 0', array([0, 1, 2, 3, 4])), ('axis 1', array([0, 1, 2]))]
